In [1]:
import pandas as pd
import re
import numpy as np
import time
import os
import pickle
import time
import pipeline as p

from collections import defaultdict

%load_ext autoreload
%autoreload 2

In [2]:
pd.set_eng_float_format(accuracy=1, use_eng_prefix=True)
team_dict = p.open_pkl('TeamDicts/team_dictionary.pkl')
url_team_dict = p.open_pkl('TeamDicts/url_team_dict.pkl')

In [5]:
os.chdir('Data/SeasonHtmls/')

In [6]:
master_df = pd.DataFrame()
# master_df = p.open_pkl('master_df.pkl')

merge_dict = defaultdict(list)
# merge_dict = p.open_pkl('merge_dict.pkl')

### Read HTML, convert to df, clean, merge

In [7]:
count = 0

for f in os.listdir():
    if f[0] != '.':
        team = url_team_dict[f[:3].lower()][2]
        year = int(f[4:8])

        if team not in merge_dict[year]:
            print('Processing ' + team, year)
            html = p.open_pkl(f)
            try:
                # covert to df & clean 
                df = p.clean_season_data(html, team, year)

                # append to master df
                master_df = master_df.append(df, ignore_index=True, sort=True)

                # add entry to tsdf_merge_track
                merge_dict[year].append(team)
                count += 1
            except ValueError as e: 
                print('Error cleaning ' + team, year, e)
                p.log_this('Error cleaning ' + team, year, e)

#         if count%10 == 0:
#             p.pkl_this('../Data/master_df.pkl', master_df)

print('Done')    

Processing NOR 2018
Processing OAK 2018
Processing STL 2018
Processing ARI 2018
Processing TAM 2018
Processing CLE 2018
Processing DET 2018
Processing DAL 2018
Processing IND 2018
Processing MIA 2018
Processing WAS 2018
Processing SDG 2018
Processing PIT 2018
Processing ATL 2018
Processing BAL 2018
Processing NWE 2018
Processing BUF 2018
Processing SEA 2018
Processing NYG 2018
Processing KAN 2018
Processing GNB 2018
Processing SFO 2018
Processing CHI 2018
Processing CIN 2018
Processing MIN 2018
Processing NYJ 2018
Processing DEN 2018
Processing HOU 2018
Processing JAX 2018
Processing CAR 2018
Processing PHI 2018
Processing TEN 2018
Done


In [8]:
master_df.head()

,1stD-O,Date,Day,Location,OT,Opp,PassY,PtsOpp,PtsTm,Record,Result,RushY,TO,TO-D,Team,Time,TotY,Week,Year,bs
0,26.0,September 9,Sun,NaN,NaN,Tampa Bay Buccaneers,432.0,48.0,40.0,0-1,L,43.0,2.0,NaN,NOR,1:00PM ET,475.0,1,2018,boxscore
1,18.0,September 16,Sun,NaN,NaN,Cleveland Browns,213.0,18.0,21.0,1-1,W,62.0,2.0,1.0,NOR,1:00PM ET,275.0,2,2018,boxscore
2,32.0,September 23,Sun,@,OT,Atlanta Falcons,391.0,37.0,43.0,2-1,W,143.0,NaN,NaN,NOR,1:00PM ET,534.0,3,2018,boxscore
3,25.0,September 30,Sun,@,NaN,New York Giants,219.0,18.0,33.0,3-1,W,170.0,NaN,2.0,NOR,4:25PM ET,389.0,4,2018,boxscore
4,27.0,October 8,Mon,NaN,NaN,Washington Redskins,349.0,19.0,43.0,4-1,W,98.0,1.0,2.0,NOR,8:15PM ET,447.0,5,2018,boxscore


In [9]:
# p.pkl_this('../Data/merge_dict.pkl', merge_dict)

In [10]:
len(master_df)

520

Check that we have 32 teams for each season:

In [11]:
for k,v in merge_dict.items():
    print(k, len(v))

2018 32


Missing team from 2016 & 2017. Figure out which one it is, and manually add it to master_df

In [23]:
team_abbrev_list = [v[2] for v in url_team_dict.values()]

In [24]:
missing_list = {}
for year in range(2007, 2018):
    missing_list[year] = np.setdiff1d(team_abbrev_list, master_df[master_df['Year'] == year]['Team'].unique()) 

In [25]:
missing_list

{2007: array([],
       dtype='<U3'), 2008: array([],
       dtype='<U3'), 2009: array([],
       dtype='<U3'), 2010: array([],
       dtype='<U3'), 2011: array([],
       dtype='<U3'), 2012: array([],
       dtype='<U3'), 2013: array([],
       dtype='<U3'), 2014: array([],
       dtype='<U3'), 2015: array([],
       dtype='<U3'), 2016: array(['PIT'],
       dtype='<U3'), 2017: array(['PIT'],
       dtype='<U3')}

In [28]:
season_html_2016 = p.open_pkl('../season_html_2016.pkl')
html = season_html_2016['PIT-2016']
team = 'PIT'
year = 2016
df = p.clean_season_data(html, team, year)

In [30]:
# append to master df
master_df = master_df.append(df, ignore_index=True, sort=True)

# add entry to tsdf_merge_track
merge_dict[year].append(team)

In [31]:
season_html_2017 = p.open_pkl('../season_html_2017.pkl')
html = season_html_2017['PIT-2017']
team = 'PIT'
year = 2017
df = p.clean_season_data(html, team, year)

In [33]:
# append to master df
master_df = master_df.append(df, ignore_index=True, sort=True)

# add entry to tsdf_merge_track
merge_dict[year].append(team)

In [34]:
len(master_df)

5874

In [40]:
for k,v in merge_dict.items():
    print(k, len(v))

2016 32
2017 32
2007 32
2008 32
2009 32
2010 32
2011 32
2012 32
2013 32
2014 32
2015 32


In [13]:
# p.pkl_this('../master_df_2018.pkl', master_df)
p.pkl_this('../master_df.pkl', master_df)
p.pkl_this('../merge_dict.pkl', merge_dict)

In [14]:
len(master_df['Team'].unique())

32